In [2]:
# ===============================
# 1. Importar librerías y artefactos
# ===============================
import torch
import pickle
from pathlib import Path
from transformers import BertTokenizer, BertForSequenceClassification

# ===============================
# 2. Cargar modelo y tokenizer
# ===============================
ROOT_DIR = Path("..").resolve()
MODEL_DIR = ROOT_DIR / "models"

tokenizer = BertTokenizer.from_pretrained(str(MODEL_DIR / "tokenizer"), local_files_only=True)
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=4)
model.load_state_dict(torch.load(MODEL_DIR / "modelo_bert_finetuned.pth", map_location="cpu"))
model.eval()

with open(MODEL_DIR / "label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# ===============================
# 3. Textos de prueba
# ===============================
textos_prueba = [
    "Quiero saber más sobre los servicios disponibles.",
    "Gracias por la excelente atención brindada.",
    "El sistema falló y no pude completar mi solicitud.",
    "Estoy inconforme con el trato recibido."
]

# ===============================
# 4. Predecir etiquetas
# ===============================
inputs = tokenizer(
    textos_prueba,
    truncation=True,
    padding="max_length",
    max_length=128,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = model(**inputs)
    _, predicciones = torch.max(outputs.logits, dim=1)

# ===============================
# 5. Mostrar resultados
# ===============================
clases = label_encoder.inverse_transform(predicciones.numpy())

for i, texto in enumerate(textos_prueba):
    print(f"📝 Texto: {texto}")
    print(f"🔎 Clasificación: {clases[i]}")
    print("-" * 50)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📝 Texto: Quiero saber más sobre los servicios disponibles.
🔎 Clasificación: Solicitud de información
--------------------------------------------------
📝 Texto: Gracias por la excelente atención brindada.
🔎 Clasificación: Retroalimentación positiva
--------------------------------------------------
📝 Texto: El sistema falló y no pude completar mi solicitud.
🔎 Clasificación: Solicitud de información
--------------------------------------------------
📝 Texto: Estoy inconforme con el trato recibido.
🔎 Clasificación: Retroalimentación positiva
--------------------------------------------------
